## Import Necessary Libraries

In [1]:
## Import necessary libraries
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote, unquote
from datetime import timedelta
from scipy.interpolate import interp1d
from scipy.fftpack import fft
from sklearn.preprocessing import MinMaxScaler

## Import libraries for the model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import f1_score, classification_report

## Set path for saving model training results  
import os
os.makedirs('./result', exist_ok=True)

## Set Cuda for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Set random seed
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# Set seed
seed_val = 77
set_seed(seed_val)

cuda


## Selecting Data Columns
* Tag names are loaded in sequential order.
* The process of selecting the required tag names from the tag name list.

In [2]:
# Function to display tag names
def show_column(URL):
    
    # Load tag name data
    df = pd.read_csv(URL)
    
    # Convert to list format
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## Set parameters for displaying tag names
table = 'rotor'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get-tag-names.tql?table={table}'

## Generate tag name list  
name = show_column(NAME_URL)

In [4]:
name

['g1_sensor1_normal',
 'g1_sensor1_type1',
 'g1_sensor1_type2',
 'g1_sensor1_type3',
 'g1_sensor2_normal',
 'g1_sensor2_type1',
 'g1_sensor2_type2',
 'g1_sensor2_type3',
 'g1_sensor3_normal',
 'g1_sensor3_type1',
 'g1_sensor3_type2',
 'g1_sensor3_type3',
 'g1_sensor4_normal',
 'g1_sensor4_type1',
 'g1_sensor4_type2',
 'g1_sensor4_type3',
 'g2_sensor1_normal',
 'g2_sensor1_type1',
 'g2_sensor1_type2',
 'g2_sensor1_type3',
 'g2_sensor2_normal',
 'g2_sensor2_type1',
 'g2_sensor2_type2',
 'g2_sensor2_type3',
 'g2_sensor3_normal',
 'g2_sensor3_type1',
 'g2_sensor3_type2',
 'g2_sensor3_type3',
 'g2_sensor4_normal',
 'g2_sensor4_type1',
 'g2_sensor4_type2',
 'g2_sensor4_type3']

## Converting TAG Name Format
* After checking all the Tag Names from the rotor dataset in the previous step, extract only the columns to be used and convert them into parameter format.
* Use tag names related to the g1

In [5]:
# Set the desired tag names
tags = name[:16]

# Wrap each item in the list with single quotes and separate with commas
tags_ = ",".join(f"'{tag}'" for tag in tags)

# Check the selected tag names
print(tags_)

'g1_sensor1_normal','g1_sensor1_type1','g1_sensor1_type2','g1_sensor1_type3','g1_sensor2_normal','g1_sensor2_type1','g1_sensor2_type2','g1_sensor2_type3','g1_sensor3_normal','g1_sensor3_type1','g1_sensor3_type2','g1_sensor3_type3','g1_sensor4_normal','g1_sensor4_type1','g1_sensor4_type2','g1_sensor4_type3'


## Load Rotor Dataset
* Load the entire dataset upon data loading.

    * Label description:

        * normal: Normal
        * type1: Rotational imbalance on Disk 2 (bolt and nut attached at the 270-degree position)
        * type2: Support imbalance on Support 4
        * type3: Combination of Type 1 and Type 2

In [6]:
# Label mapping

# Label mapping dictionary
label_mapping = {
    'normal': 0,
    'type1': 1,
    'type2': 2,
    'type3': 3
}

# Function to extract labels from column names
def get_label(column_name):
    column_name = str(column_name)
    for key in label_mapping.keys():
        if key in column_name:
            return label_mapping[key]
    return None 

In [7]:
# Data loading function
def data_load(table, name, start_time, end_time, timeformat):
    
    # Load data  
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')
    
    # Convert to data grouped by the time
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # Set TIME column
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')
    
    # Create an empty DataFrame
    data_result = pd.DataFrame()

    # Interpolate data for each tag name
    for i in range(len(df.columns[1:])):
        
        # Set time
        start = pd.to_datetime(unquote(start_time))
        end = pd.to_datetime(unquote(end_time))
        
        data_ = df.iloc[ : , [0] + list(range(i+1, i+2))].dropna()
        
        # Create a new time range for interpolation (1000 points for each second)
        # In this case, the original data was measured at 1 ms intervals, so we create 1000 points at 1-second intervals
        # Generate range from 0 to 140 -> 140,000
        new_time_range = pd.date_range(start=start, end=end, freq='1ms')[:-1]
        new_time_range_ = pd.date_range(start=start, end=end, freq='1s')[:-1]

        # Use linear interpolation to fill in the data
        # Convert datetime to numeric (epoch time in seconds)
        time_numeric = pd.to_numeric(data_['TIME'])
        new_time_numeric = pd.to_numeric(new_time_range)

        value = data_[data_.columns[1:].values]

        # Create linear interpolation object
        interpolator = interp1d(time_numeric, value.values.reshape(-1), kind='linear', fill_value='extrapolate')
        interpolated_values = interpolator(new_time_numeric)
        interpolated_values = np.clip(interpolated_values, min(value.values), max(value.values))

        # Create DataFrame
        data_remake = pd.DataFrame(interpolated_values.reshape(-1,1000))
        data_remake['time'] = new_time_range_
        data_remake['sensor'] = f'{data_.columns[1:].item()}'

        # Specify columns to move and the new order
        cols = data_remake.columns.tolist()
        cols.insert(0, cols.pop(cols.index('time')))
        cols.insert(1, cols.pop(cols.index('sensor')))
        data_remake = data_remake[cols]

        # Append to the empty DataFrame
        data_result = pd.concat([data_result, data_remake], ignore_index=True)
        
    # Sort by time
    data_result = data_result.sort_values(by='time').reset_index(drop=True)
    
    # Apply labels to each column and create a new series
    labels = pd.Series(data_result['sensor']).map(get_label)

    # Add the label to the DataFrame
    data_result['label'] = labels.values

    # Print the count of each label
    data_result = data_result.iloc[:,2:]
        
    return data_result

In [8]:
# Data time loading function
def time_data_load(table, name, start_time, end_time, timeformat):
    
    target = 'time'
    
    # Load the data  
    df = pd.read_csv(f"http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?target={target}&table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}")
    
    # Create a dummy value column for resampling
    df['value'] = 0
    
    # Perform resampling
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    df = df.resample('1s').mean()
    
    # Remove missing values
    df = df.dropna()
    
    # Remove the dummy value column
    df = df.drop(['value'], axis=1)
    
    return df

In [9]:
# Time update function
# Update start and end times based on batch size
def update_time(time_df, start_time, batch_size):
    
    # Calculate how many data points need to be loaded
    time = int(batch_size / 16)
    
    # Check the index number of the current time
    # If not found, set to the first index as there is no data for the current time
    try:
        index_now = time_df.index.get_loc(start_time)
    except KeyError:
        index_now = 0
    
    # Set the end time for the batch data based on the current time 
    end_time_ = str(time_df.index[index_now + time])
    
    # Set the index number for the next start time
    index_next = index_now + time
    
    # Set the next start time
    next_start_time_ = str(time_df.index[index_next])
    
    # URL encoding
    start_time_ = quote(start_time)
    end_time_ = quote(end_time_)
    next_start_time_ = quote(next_start_time_)
    
    return start_time_, end_time_, next_start_time_, index_next

## Data Preprocessing
   * 1 Hanning Window
   * 2 FFT 
   * 3 MinMaxScaling -> Apply during training

### 1. Applying Hanning Window

In [10]:
# Hanning window function setup 
def set_hanning_window(sample_rate, df):
    
    # Generate Hanning window
    hanning_window = np.hanning(sample_rate)

    # Apply Hanning window to each row
    df_windowed = df.multiply(hanning_window, axis=1)
    
    return df_windowed

### 2. Applying FFT (Fast Fourier Transform)

In [11]:
# FFT transformation function
def change_fft(sample_rate, df):
    # Total number of samples in the signal
    N = sample_rate
    
    fft_results = np.zeros((df.shape[0], N // 2 + 1), dtype=float)
    
    # Apply FFT to each row
    for i in range(df.shape[0]):
        
        # Calculate FFT for each row
        yf = fft(df.iloc[i].values)
        
        # Compute the absolute value of the FFT results and normalize (only the meaningful part)
        fft_results[i] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    # Convert FFT results to a DataFrame
    fft_df = pd.DataFrame(fft_results)
    
    return fft_df

## Model Configuration
* Using ResNet1d model.

In [12]:
## ResNet 1D Model Setup
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

class ResNet1D(nn.Module):
    def __init__(self, block, layers, num_classes=4):
        super(ResNet1D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [13]:
# Model configuration parameters
# Learning rate
learning_rate = 0.01

# Model configuration
model = ResNet1D(ResidualBlock, [2, 2, 2, 2], num_classes=4).to(device)

# Configure loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Check the model architecture
print(model)

ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), paddi

## Model Training

* Save the model with the Best F1 Score based on the validation data during training.

In [15]:
# Model training function
def train(table, name, timeformat, model, batch_size, epochs, scaler, time_df_train, time_df_valid, sample_rate):
    
    # Initialize training loss
    train_loss = []
    train_acc = []

    # Initialize best F1 Score value
    best_f1= -np.inf

    # Start model training
    for epoch in epochs:
        
        model.train()

        running_loss = 0.0
        total_step = 0
        correct = 0
        total=0

        # Set initial start time
        start_time_ = str(time_df_train.index[0])

        # Set end time
        end_time_train = str(time_df_train.index[-1])

        # Use a while loop to call data
        while start_time_ < end_time_train:
            
            # Set the time for loading data based on the batch size 
            start_time_, end_time_, next_start_time_, index_next= update_time(time_df_train, start_time_, batch_size)
            
            # Load batch data 
            data = data_load(table, name, start_time_, end_time_, timeformat)
            
            # Apply Hanning window
            data_ = set_hanning_window(sample_rate, data.iloc[:,:-1])
            
            # Apply FFT
            data_  = change_fft(sample_rate, data_ )
            
            # Apply MinMax scaler
            data_ = scaler.fit_transform(data_)
            
            # Setting up DataFrame + label
            data_ = pd.DataFrame(data_)
            data_['label'] = data['label'].values
            
            # Data Random Shuffle
            data_ = data_.sample(frac=1).reset_index(drop=True)
            
            # Print if the loaded data is empty 
            if len(data_) == 0:
                print("No data available.")
            
            # Input the data into the model when it accumulates to the batch size
            if len(data_) == batch_size:
                
                # Check total batch count  
                total_step = total_step + 1
                
                # Convert data to numpy arrays
                input_data = np.array(data_.iloc[:,:-1]).reshape(batch_size, 1 , -1)
                label = np.array(data_.iloc[:,-1:])

                # Convert data to Tensor
                input_data = torch.tensor(input_data, dtype=torch.float32).to(device).float()
                label = torch.tensor(label).to(device).long().squeeze()

                # Optimize the optimizer
                optimizer.zero_grad()
                
                # Input to the model
                outputs = model(input_data)
                
                # Calculate loss
                loss = criterion(outputs, label)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # Set label predictions 
                _,pred = torch.max(outputs, dim=1)
                correct += torch.sum(pred==label).item()
                total += label.size(0)
                
                # Reset batch data
                data_ = 0
                
            # Set the next start time     
            start_time_ = unquote(next_start_time_)
                
            # Prevent fetching beyond the last time
            if index_next + (int(batch_size /16)) >= len(time_df_train):
                break
            
        train_acc.append(100 * correct / total)
        train_loss.append(running_loss/total_step)
        print(f'\ntrain loss: {np.mean(train_loss)}, train acc: {(100 * correct / total):.4f}')

        # Perform validation at the end of each epoch and save the model with the best performance
        with torch.no_grad():
            
            model.eval()
            
            preds_ = []
            targets_ = []
                
            # Set initial start time
            start_time_v = str(time_df_valid.index[0])
            
            # Set end time
            end_time_valid = str(time_df_valid.index[-1])
            
            # Use a while loop to call data 
            while start_time_v < end_time_valid:
                
                # Set the time for loading data based on the batch size
                start_time_v, end_time_v, next_start_time_v, index_next_v = update_time(time_df_valid, start_time_v, batch_size)
                
                # Load batch data 
                data_v = data_load(table, name, start_time_v, end_time_v, timeformat)
                
                # Apply Hanning window
                data_ = set_hanning_window(sample_rate, data_v.iloc[:,:-1])
                
                # Apply FFT 
                data_  = change_fft(sample_rate, data_ )
                
                # Apply MinMax scaler 
                data_ = scaler.fit_transform(data_)
                
                # Setting up DataFrame + label
                data_ = pd.DataFrame(data_)
                data_['label'] = data_v['label'].values
                
                # Data Random Shuffle
                data_ = data_.sample(frac=1).reset_index(drop=True)
                
                # Print if the loaded data is empty
                if len(data_) == 0:
                    print("No data available.")
                
                # Input the data into the model when it accumulates to the batch size
                if len(data_) == batch_size:
                    
                    # Convert data to numpy arrays
                    input_data_v = np.array(data_.iloc[:,:-1]).reshape(batch_size, 1 , -1)
                    label_v = np.array(data_.iloc[:,-1:])

                    # Convert data to Tensor
                    input_data_v = torch.tensor(input_data_v, dtype=torch.float32).to(device).float()
                    label_v = torch.tensor(label_v).to(device).long().squeeze()
                    
                    # Input to the model
                    outputs_v = model(input_data_v)
                    
                    # Set label predictions 
                    _,pred_v = torch.max(outputs_v, dim=1)
                    target_v = label_v.view_as(pred_v)
      
                    preds_.append(pred_v)
                    targets_.append(target_v)
                    
                    # Reset batch data
                    data_ = 0
                    
                # Set the next start time    
                start_time_v = unquote(next_start_time_v)
                
                # Prevent fetching beyond the last time
                if index_next_v + (int(batch_size /16)) >= len(time_df_valid):
                    break
                    
            # Combine predictions and labels collected from all batches
            preds_v = torch.cat(preds_).detach().cpu().numpy()
            targets_v = torch.cat(targets_).detach().cpu().numpy()
            
            f1score = f1_score(targets_v, preds_v,  average='macro')
            if best_f1 < f1score:
                best_f1 = f1score
                # Save the best model 
                with open("./result/rotor_1d_ResNet_New_Batch.txt", "a") as text_file:
                    print('epoch=====',epoch, file=text_file)
                    print(classification_report(targets_v, preds_v, digits=4), file=text_file)
                torch.save(model, f'./result/rotor_1d_ResNet_New_Batch.pt') 
            epochs.set_postfix_str(f"epoch = {epoch},  f1_score = {f1score}, best_f1 = {best_f1}")
               
    return model

In [16]:
########################################### Training Parameter Settings ################################################
# Set tag table name
table = 'rotor'
# Set tag name
name = quote(tags_, safe=":/")
# Set the start time for the train data
start_time_train = '2024-01-01 00:00:00'
# Set the end time for the train data
end_time_train = '2024-01-01 00:01:39'
# Set time format
timeformat = quote('2006-01-02 15:04:05.000000')
# Set batch size
# Fix the batch size to 16 for this rotor data
batch_size = 16
# Set number of epochs
epochs = trange(30, desc='training')
# Set sample rate
sample_rate = 1000
# Set Min-Max scaler
scaler = MinMaxScaler()
# Load training time list 
time_df_train = time_data_load(table, name, quote(start_time_train), quote(end_time_train), timeformat)

########################################### validation Parameter Settings ################################################
# Set the start time for the validation data
start_time_valid = '2024-01-01 00:01:39'
# Set the end time for the validation data
end_time_valid = '2024-01-01 00:02:00'
# Load validation time list
time_df_valid = time_data_load(table, name, quote(start_time_valid), quote(end_time_valid), timeformat)

########################################### Proceed with training ################################################
train(table, name, timeformat, model, batch_size, epochs, scaler, time_df_train, time_df_valid, sample_rate)

training:   0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass



train loss: 1.202284309328819, train acc: 48.8520

train loss: 1.0962372036004553, train acc: 55.1020

train loss: 0.9918817787551556, train acc: 65.5612

train loss: 0.9008833318188483, train acc: 73.2143

train loss: 0.8220373034781339, train acc: 79.7194

train loss: 0.746996943401743, train acc: 85.3954

train loss: 0.6775784237614396, train acc: 89.6046

train loss: 0.619259966180983, train acc: 92.4745

train loss: 0.5672647098919837, train acc: 94.6429

train loss: 0.524029608508477, train acc: 94.7066

train loss: 0.48373553668216207, train acc: 97.1301

train loss: 0.44921779448016425, train acc: 97.2577

train loss: 0.4222075491724207, train acc: 96.6199

train loss: 0.3988481249123827, train acc: 96.6199

train loss: 0.37548347991300424, train acc: 98.4056

train loss: 0.3551015420195319, train acc: 98.5332

train loss: 0.3373091416777511, train acc: 98.3418

train loss: 0.32364740021720145, train acc: 96.1735

train loss: 0.3100498602162167, train acc: 97.6403

train loss:

ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), paddi

## Model Testing

In [17]:
# Model testing function
def test(table, name, timeformat, model, batch_size, sample_rate, scaler, time_df_test):

    with torch.no_grad():
        
        model.eval()
        
        # Initial settings
        preds_test = []
        target_test = []
        
        # Set the initial start time
        start_time_t = str(time_df_test.index[0])
        
        # Set the end time
        end_time_test = str(time_df_test.index[-1])
        
        # Use a while loop to call data  
        while start_time_t < end_time_test:
            
            # Set the time for loading data based on the batch size
            start_time_t, end_time_t, next_start_time_t, index_next_t = update_time(time_df_test, start_time_t, batch_size)
            
            # Load batch data
            data_v = data_load(table, name, start_time_t, end_time_t, timeformat)
            
            # Apply Hanning window
            data_ = set_hanning_window(sample_rate, data_v.iloc[:,:-1])
            
            # Apply FFT
            data_  = change_fft(sample_rate, data_ )
            
            # Apply MinMax scaler
            data_ = scaler.fit_transform(data_)
            
            # Setting up DataFrame + label
            data_ = pd.DataFrame(data_)
            data_['label'] = data_v['label'].values
            
            # Print if the loaded data is empty
            if len(data_) == 0:
                print("No data available.")
            
            # Input the data into the model when it accumulates to the batch size
            if len(data_) == batch_size:
                
                # Convert data to numpy arrays
                input_data_test = np.array(data_.iloc[:,:-1]).reshape(batch_size, 1 , -1)
                input_data_label = np.array(data_.iloc[:,-1:])
                
                # Convert data to Tensor
                input_data_test = torch.tensor(input_data_test, dtype=torch.float32).to(device).float()
                input_data_label = torch.tensor(input_data_label, dtype=torch.float32).to(device).long()
                
                # Create DataLoader
                dataset = TensorDataset(input_data_test, input_data_label)
                data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
                
                for batch_input, batch_label in data_loader:

                    # Input to the model
                    outputs_t = model(batch_input)
                    
                    # Set label predictions
                    _,pred_t = torch.max(outputs_t, dim=1)
                    targets_t = batch_label.view_as(pred_t).to(device)
                    
                    preds_test.append(pred_t)
                    target_test.append(targets_t)
                
                # Reset batch data
                data_ = []
                
            # Set the next start time   
            start_time_t = unquote(next_start_time_t) 
            
            # Prevent fetching beyond the last time
            if index_next_t + (int(batch_size /16)) >= len(time_df_test):
                break
            
    # Combine predictions and labels collected from all batches
    preds_test = torch.cat(preds_test).detach().cpu().numpy()
    target_test = torch.cat(target_test).detach().cpu().numpy()

    # Create Result DataFrame
    final_df = pd.DataFrame(target_test, columns=['label'])
    final_df['pred'] = target_test
    
    return final_df

In [18]:
########################################### Test Parameter Settings ################################################
# Load the best model
model_ = torch.load(f'./result/rotor_1d_ResNet_New_Batch.pt') 
# Set the start time for the test data
start_time_test = '2024-01-01 00:01:59'
# Set the end time for the test data
end_time_test = '2024-01-01 00:02:20'
# Load the test time list
time_df_test = time_data_load(table, name, quote(start_time_test), quote(end_time_test), timeformat)

######################################## Proceed with testing #############################################
final_df = test(table, name, timeformat, model_, batch_size, sample_rate, scaler, time_df_test)

c:\Users\MACH-DE-28\anaconda3\envs\test\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


## Model Performance Evaluation

In [19]:
print(classification_report(final_df['label'].values, final_df['pred'].values))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        80
           1       1.00      1.00      1.00        80
           2       1.00      1.00      1.00        80
           3       1.00      1.00      1.00        80

    accuracy                           1.00       320
   macro avg       1.00      1.00      1.00       320
weighted avg       1.00      1.00      1.00       320

